# Importing Packages and creating universally helpful things

In [1]:
import numpy as np
import pandas as pd
import re
import requests
from collections import Counter
import bs4 as bs
import unicodedata
mdict = {"January":"1", "February":"2", "March":"3", "April":"4", "May":"5", "June":"6", "July":"7",\
         "August":"8", "September":"9", "October":"10", "November":"11", "December":"12"}
import us
STATES_DICT = us.states.mapping('name', 'abbr')

def strip_accents(text):
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass
    text = unicodedata.normalize('NFD', text)
    text = text.encode('ascii', 'ignore')
    text = text.decode("utf-8")
    return str(text)
def stripper(begins):
    begins = [strip_accents(boo) for boo in begins]
    begins = [re.sub(r"%C3%BA","u", u) for u in begins]
    begins = [re.sub(r"%C3%A9","e", e) for e in begins]
    begins = [re.sub(r"%C3%AD","i", i) for i in begins]
    begins = [re.sub(r"%C3%A1","a", a) for a in begins]
    begins = [re.sub("%27", "'", nra) for nra in begins]
    #begins = [re.sub(" [A-Z]\. ", " ", nra) for nra in begins]
    return(begins)



# 112th Congress (from January 3, 2011, until January 3, 2013)

In [2]:
source = requests.get("https://en.wikipedia.org/wiki/112th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link.get('href')) for link in soup.find_all('a')]
begins = stripper(begins)
ends = []
for string in begins:
    if re.match(r"/wiki", string):
        ends.append(re.sub(r"/wiki/","",string))
sens112 = ends[237:390]
reps112 = ends[400:1331]
####
####
####
sstate = []
schamber = []
sname = []
rstate = []
rchamber = []
rname = []
for sen in sens112:
    if re.match(r"List_of.*", sen):
        state = re.sub(r"^List_of_United_States_Senators_from_","",sen)
    else:
        name = re.sub(r"_"," ", sen)
        name = re.sub(r"\(.*\)","",name)
        sname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        sstate.append(state)
        schamber.append("Senate")
for dis in reps112:
    if re.match(r"List_of.*", dis):
        state = re.sub(r"^List_of_United_States_Representatives_from_","",dis)
    #elif re.match(r".*_congressional_district$", dis):
    #    house115.append(state)
    #    temp = re.sub(r"_congressional_district$","",dis)
    #    house115.append("DISTRICT: " + re.sub(r"^.*[^_]+_","", temp))
    elif not re.match(r".*_congressional_district$", dis):
        name = re.sub(r"_"," ",dis)
        name = re.sub(r"\(.*\)","",name)
        rname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        rstate.append(state)
        rchamber.append("House")
rstates112 = pd.concat([pd.Series(rname), pd.Series(rchamber), pd.Series(rstate)], axis = 1)
rstates112.columns = ["Name", "Chamber", "State"] 
sstates112 = pd.concat([pd.Series(sname), pd.Series(schamber), pd.Series(sstate)], axis = 1)
sstates112.columns = ["Name", "Chamber", "State"] 

In [3]:
sstates112.head()

,Name,Chamber,State
0,Jeff Sessions,Senate,Alabama
1,Richard Shelby,Senate,Alabama
2,Mark Begich,Senate,Alaska
3,Lisa Murkowski,Senate,Alaska
4,Jon Kyl,Senate,Arizona


In [4]:
sstates112.tail()

,Name,Chamber,State
98,Jay Rockefeller,Senate,West_Virginia
99,Herb Kohl,Senate,Wisconsin
100,Ron Johnson,Senate,Wisconsin
101,John Barrasso,Senate,Wyoming
102,Mike Enzi,Senate,Wyoming


In [5]:
rstates112.head()

,Name,Chamber,State
0,Jo Bonner,House,Alabama
1,Martha Roby,House,Alabama
2,Mike D. Rogers,House,Alabama
3,Robert Aderholt,House,Alabama
4,Mo Brooks,House,Alabama


In [6]:
rstates112.tail()

,Name,Chamber,State
440,Jim Sensenbrenner,House,Wisconsin
441,Tom Petri,House,Wisconsin
442,Sean Duffy,House,Wisconsin
443,Reid Ribble,House,Wisconsin
444,Cynthia Lummis,House,Wyoming


In [7]:
# 112th Congress (from January 3, 2011, until January 3, 2013)

In [8]:
source = requests.get("https://en.wikipedia.org/wiki/112th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link) for link in soup.find_all('li')]
begins = stripper(begins)
#begins = [re.sub(r"\'","a",a) for a in begins]
sens112 = begins[221:324]
reps112 = begins[375:824]
syear = ["<BLANK>" for x in range(len(sens112))]
sday = ["<BLANK>" for x in range(len(sens112))]
smonth = ["<BLANK>" for x in range(len(sens112))]
sparty = ["<BLANK>" for x in range(len(sens112))]
sname = ["<BLANK>" for x in range(len(sens112))]
sstate = ["<BLANK>" for x in range(len(sens112))]
##########
ryear = ["<BLANK>" for x in range(len(reps112))]
rday = ["<BLANK>" for x in range(len(reps112))]
rmonth = ["<BLANK>" for x in range(len(reps112))]
rparty = ["<BLANK>" for x in range(len(reps112))]
rname = ["<BLANK>" for x in range(len(reps112))]
rstate = ["<BLANK>" for x in range(len(reps112))]
rdistrict = ["<BLANK>" for x in range(len(reps112))]
hmm = [re.sub(r"<[^>]+>","",sen) for sen in sens112]
for sen in range(len(hmm)):
    z = re.match("^3\..*",hmm[sen]) #class 3 begins this congress
    if z:
        syear[sen] = "2010" #election year
        smonth[sen] = "11" #election month
        sday[sen] = "2" #election day
        name = re.sub("^3\. ","",hmm[sen]) #Senate cycle according to wikipedia metadata
        sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(sname[sen],"",name)
        party = re.findall(r"\([^ ]+\)", party)[0]
        sparty[sen] = re.sub(r"[^A-Z]+","",party)
for sen in range(len(hmm)):
    if syear[sen] == "<BLANK>":
        if re.match("^[^0-9].*",hmm[sen]):
            step2 = re.findall(r"[A-Za-z]+ [0-9]+, \d\d\d\d",hmm[sen])[0]
            sday[sen] = re.sub(r",","",re.findall(r"[0-9]+,",step2)[0])
            syear[sen] = re.findall(r"\d\d\d\d", step2)[0]
            smonth[sen] = mdict[re.sub(r"[0-9, ]+","", step2)]
            name = re.sub(r"\[[^ ]+\]","",hmm[sen])
            sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
            party = re.sub(sname[sen],"",name)
            sparty[sen] = re.sub(r"[^A-Z]+","",party)
syear = [x for x in syear if x!= "<BLANK>"]
smonth = [x for x in smonth if x!= "<BLANK>"]
sday = [x for x in sday if x!= "<BLANK>"]
sname = [x for x in sname if x!= "<BLANK>"]
sparty = [x[0] for x in sparty if x!= "<BLANK>"]
schamber = ["Senate" for x in sparty if x!="<BLANK>"]
df = pd.DataFrame({'year': syear,
                       'month': smonth,
                       'day': sday})
sdate = pd.to_datetime(df)
senate112 = pd.concat([sdate, pd.Series(sparty), pd.Series(sname), pd.Series(schamber)], axis = 1)
senate112.columns = ["Date", "Party", "Name", "Chamber"]
#######
#######
#######
hmm = [re.sub(r"<[^>]+>","",rep) for rep in reps112]
hmm = [re.sub("[Aa]t\-large\.","0.",rep) for rep in hmm]
for rep in range(len(hmm)):
    z = re.match("^[0-9]+\..*",hmm[rep])
    if z:
        district = re.findall(r"^[0-9]+", hmm[rep])[0]
        rdistrict[rep] = district
        name = re.sub(rdistrict[rep] + ". ","", hmm[rep])
        rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(rname[rep] + " ","",name)
        party = re.findall(r"^\([^ ]+\)",party)[0]
        rparty[rep] = re.sub(r"[\(\)]","", party)
        ryear[rep] = "2010" #election year
        rmonth[rep] = "11" #election month
        rday[rep] = "2" #election day   
    else:
        if not re.search(".*Vacant.*", hmm[rep]):
            rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", hmm[rep])[0]
            rdistrict[rep] = district
            party = re.sub(rname[rep] + " ","", hmm[rep])
            party = re.findall(r"^\([^ ]+\)",party)[0]
            rparty[rep] = re.sub(r"[\(\)]","", party)
            date = re.sub("^.*from ","",hmm[rep])
            ryear[rep] = re.findall(r"\d\d\d\d",date)[0]
            date = re.sub(ryear[rep],"",date)
            date = re.sub(r",.*$","",date)
            rday[rep] = re.findall(r"[0-9]+",date)[0]
            date = re.sub(" " + rday[rep],"",date)
            rmonth[rep] = mdict[date]
ryear = [x for x in ryear if x!= "<BLANK>"]
rmonth = [x for x in rmonth if x!= "<BLANK>"]
rday = [x for x in rday if x!= "<BLANK>"]
rname = [x for x in rname if x!= "<BLANK>"]
rparty = [x[0] for x in rparty if x!= "<BLANK>"]
rchamber = ["House" for x in rparty if x!= "<BLANK>"]
rdistrict = [x for x in rdistrict if x!= "<BLANK>"]
df = pd.DataFrame({'year': ryear,
                       'month': rmonth,
                       'day': rday})
rdate = pd.to_datetime(df)
house112 = pd.concat([rdate, pd.Series(rparty), pd.Series(rname), pd.Series(rchamber), pd.Series(rdistrict)], axis = 1)
house112.columns = ["Date", "Party", "Name", "Chamber", "District"]  

In [9]:
senate112.head()

,Date,Party,Name,Chamber
0,2010-11-02,R,Richard Shelby,Senate
1,2010-11-02,R,Lisa Murkowski,Senate
2,2010-11-02,R,John McCain,Senate
3,2010-11-02,R,John Boozman,Senate
4,2010-11-02,D,Barbara Boxer,Senate


In [10]:
senate112.tail()

,Date,Party,Name,Chamber
32,2010-11-02,R,John Thune,Senate
33,2010-11-02,R,Mike Lee,Senate
34,2010-11-02,D,Patrick Leahy,Senate
35,2010-11-02,D,Patty Murray,Senate
36,2010-11-02,R,Ron Johnson,Senate


In [11]:
house112.head()

,Date,Party,Name,Chamber,District
0,2010-11-02,R,Jo Bonner,House,1
1,2010-11-02,R,Martha Roby,House,2
2,2010-11-02,R,Mike Rogers,House,3
3,2010-11-02,R,Robert Aderholt,House,4
4,2010-11-02,R,Mo Brooks,House,5


In [12]:
house112.tail()

,Date,Party,Name,Chamber,District
440,2010-11-02,R,Jim Sensenbrenner,House,5
441,2010-11-02,R,Tom Petri,House,6
442,2010-11-02,R,Sean Duffy,House,7
443,2010-11-02,R,Reid Ribble,House,8
444,2010-11-02,R,Cynthia Lummis,House,0


# 113th Congress (from January 3, 2013, to January 3, 2015)

In [13]:
source = requests.get("https://en.wikipedia.org/wiki/113th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link.get('href')) for link in soup.find_all('a')]
begins = stripper(begins)
ends = []
for string in begins:
    if re.match(r"/wiki", string):
        ends.append(re.sub(r"/wiki/","",string))
sens113 = ends[236:391]
reps113 = ends[402:1331]
####
####
####
sstate = []
schamber = []
sname = []
rstate = []
rchamber = []
rname = []
for sen in sens113:
    if re.match(r"List_of.*", sen):
        state = re.sub(r"^List_of_United_States_Senators_from_","",sen)
    else:
        name = re.sub(r"_"," ", sen)
        name = re.sub(r"\(.*\)","",name)
        sname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        sstate.append(state)
        schamber.append("Senate")
for dis in reps113:
    if re.match(r"List_of.*", dis):
        state = re.sub(r"^List_of_United_States_Representatives_from_","",dis)
    #elif re.match(r".*_congressional_district$", dis):
    #    house115.append(state)
    #    temp = re.sub(r"_congressional_district$","",dis)
    #    house115.append("DISTRICT: " + re.sub(r"^.*[^_]+_","", temp))
    elif not re.match(r".*_congressional_district$", dis):
        name = re.sub(r"_"," ",dis)
        name = re.sub(r"\(.*\)","",name)
        rname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        rstate.append(state)
        rchamber.append("House")
rstates113 = pd.concat([pd.Series(rname), pd.Series(rchamber), pd.Series(rstate)], axis = 1)
rstates113.columns = ["Name", "Chamber", "State"] 
sstates113 = pd.concat([pd.Series(sname), pd.Series(schamber), pd.Series(sstate)], axis = 1)
sstates113.columns = ["Name", "Chamber", "State"] 

In [14]:
sstates113.head()

,Name,Chamber,State
0,Jeff Sessions,Senate,Alabama
1,Richard Shelby,Senate,Alabama
2,Mark Begich,Senate,Alaska
3,Lisa Murkowski,Senate,Alaska
4,Jeff Flake,Senate,Arizona


In [15]:
sstates113.tail()

,Name,Chamber,State
100,Jay Rockefeller,Senate,West_Virginia
101,Tammy Baldwin,Senate,Wisconsin
102,Ron Johnson,Senate,Wisconsin
103,John Barrasso,Senate,Wyoming
104,Mike Enzi,Senate,Wyoming


In [16]:
rstates113.head()

,Name,Chamber,State
0,Jo Bonner,House,Alabama
1,Bradley Byrne,House,Alabama
2,Martha Roby,House,Alabama
3,Mike D. Rogers,House,Alabama
4,Robert Aderholt,House,Alabama


In [17]:
rstates113.tail()

,Name,Chamber,State
439,Jim Sensenbrenner,House,Wisconsin
440,Tom Petri,House,Wisconsin
441,Sean Duffy,House,Wisconsin
442,Reid Ribble,House,Wisconsin
443,Cynthia Lummis,House,Wyoming


In [18]:
# 113th Congress (from January 3, 2013, to January 3, 2015)

In [19]:
source = requests.get("https://en.wikipedia.org/wiki/113th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link) for link in soup.find_all('li')]
begins = stripper(begins)
sens113 = begins[237:342]
reps113 = begins[393:837]
syear = ["<BLANK>" for x in range(len(sens113))]
sday = ["<BLANK>" for x in range(len(sens113))]
smonth = ["<BLANK>" for x in range(len(sens113))]
sparty = ["<BLANK>" for x in range(len(sens113))]
sname = ["<BLANK>" for x in range(len(sens113))]
sstate = ["<BLANK>" for x in range(len(sens113))]
##########
ryear = ["<BLANK>" for x in range(len(reps113))]
rday = ["<BLANK>" for x in range(len(reps113))]
rmonth = ["<BLANK>" for x in range(len(reps113))]
rparty = ["<BLANK>" for x in range(len(reps113))]
rname = ["<BLANK>" for x in range(len(reps113))]
rstate = ["<BLANK>" for x in range(len(reps113))]
rdistrict = ["<BLANK>" for x in range(len(reps113))]
hmm = [re.sub(r"<[^>]+>","",sen) for sen in sens113]
for sen in range(len(hmm)):
    z = re.match("^1\..*",hmm[sen]) #class 2 begins this congress
    if z:
        syear[sen] = "2012" #election year
        smonth[sen] = "11" #election month
        sday[sen] = "6" #election day
        name = re.sub("^1\. ","",hmm[sen]) #Senate cycle according to wikipedia metadata
        sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(sname[sen],"",name)
        party = re.findall(r"\([^ ]+\)", party)[0]
        sparty[sen] = re.sub(r"[^A-Z]+","",party)
for sen in range(len(hmm)):
    if syear[sen] == "<BLANK>":
        if re.match("^[^0-9].*",hmm[sen]):
            step2 = re.findall(r"[A-Za-z]+ [0-9]+, \d\d\d\d",hmm[sen])[0]
            sday[sen] = re.sub(r",","",re.findall(r"[0-9]+,",step2)[0])
            syear[sen] = re.findall(r"\d\d\d\d", step2)[0]
            smonth[sen] = mdict[re.sub(r"[0-9, ]+","", step2)]
            name = re.sub(r"\[[^ ]+\]","",hmm[sen])
            sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
            party = re.sub(sname[sen],"",name)
            sparty[sen] = re.sub(r"[^A-Z]+","",party)
syear = [x for x in syear if x!= "<BLANK>"]
smonth = [x for x in smonth if x!= "<BLANK>"]
sday = [x for x in sday if x!= "<BLANK>"]
sname = [x for x in sname if x!= "<BLANK>"]
sparty = [x[0] for x in sparty if x!= "<BLANK>"]
schamber = ["Senate" for x in sparty if x!= "<BLANK>"]
df = pd.DataFrame({'year': syear,
                       'month': smonth,
                       'day': sday})
sdate = pd.to_datetime(df)
senate113 = pd.concat([sdate, pd.Series(sparty), pd.Series(sname), pd.Series(schamber)], axis = 1)
senate113.columns = ["Date", "Party", "Name", "Chamber"]
#######
#######
#######
hmm = [re.sub(r"<[^>]+>","",rep) for rep in reps113]
hmm = [re.sub("[Aa]t\-large\.","0.",rep) for rep in hmm]
for rep in range(len(hmm)):
    z = re.match("^[0-9]+\..*",hmm[rep])
    if z:
        district = re.findall(r"^[0-9]+", hmm[rep])[0]
        rdistrict[rep] = district
        name = re.sub(rdistrict[rep] + ". ","", hmm[rep])
        rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(rname[rep] + " ","",name)
        party = re.findall(r"^\([^ ]+\)",party)[0]
        rparty[rep] = re.sub(r"[\(\)]","", party)
        ryear[rep] = "2012" #election year
        rmonth[rep] = "11" #election month
        rday[rep] = "6" #election day   
    else:
        if not re.search(".*Vacant.*", hmm[rep]):
            rdistrict[rep] = district
            rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", hmm[rep])[0]
            party = re.sub(rname[rep] + " ","", hmm[rep])
            party = re.findall(r"^\([^ ]+\)",party)[0]
            rparty[rep] = re.sub(r"[\(\)]","", party)
            date = re.sub("^.*from ","",hmm[rep])
            ryear[rep] = re.findall(r"\d\d\d\d",date)[0]
            date = re.sub(ryear[rep],"",date)
            date = re.sub(r",.*$","",date)
            rday[rep] = re.findall(r"[0-9]+",date)[0]
            date = re.sub(" " + rday[rep],"",date)
            rmonth[rep] = mdict[date]
ryear = [x for x in ryear if x!= "<BLANK>"]
rmonth = [x for x in rmonth if x!= "<BLANK>"]
rday = [x for x in rday if x!= "<BLANK>"]
rname = [x for x in rname if x!= "<BLANK>"]
rparty = [x[0] for x in rparty if x!= "<BLANK>"]
rchamber = ["House" for x in rparty if x!= "<BLANK>"]
rdistrict = [x for x in rdistrict if x!= "<BLANK>"]
df = pd.DataFrame({'year': ryear,
                       'month': rmonth,
                       'day': rday})
rdate = pd.to_datetime(df)
house113 = pd.concat([rdate, pd.Series(rparty), pd.Series(rname), pd.Series(rchamber), pd.Series(rdistrict)], axis = 1)
house113.columns = ["Date", "Party", "Name", "Chamber", "District"]  

In [20]:
senate113.head()

,Date,Party,Name,Chamber
0,2012-11-06,R,Jeff Flake,Senate
1,2012-11-06,D,Dianne Feinstein,Senate
2,2012-11-06,D,Chris Murphy,Senate
3,2012-11-06,D,Tom Carper,Senate
4,2012-11-06,D,Bill Nelson,Senate


In [21]:
senate113.tail()

,Date,Party,Name,Chamber
33,2012-11-06,D,Tim Kaine,Senate
34,2012-11-06,D,Maria Cantwell,Senate
35,2012-11-06,D,Joe Manchin,Senate
36,2012-11-06,D,Tammy Baldwin,Senate
37,2012-11-06,R,John Barrasso,Senate


In [22]:
house113.head()

,Date,Party,Name,Chamber,District
0,2012-11-06,R,Jo Bonner,House,1
1,2014-01-07,R,Bradley Byrne,House,1
2,2012-11-06,R,Martha Roby,House,2
3,2012-11-06,R,Mike Rogers,House,3
4,2012-11-06,R,Robert Aderholt,House,4


In [23]:
house113.tail()

,Date,Party,Name,Chamber,District
439,2012-11-06,R,Jim Sensenbrenner,House,5
440,2012-11-06,R,Tom Petri,House,6
441,2012-11-06,R,Sean Duffy,House,7
442,2012-11-06,R,Reid Ribble,House,8
443,2012-11-06,R,Cynthia Lummis,House,0


# 114th Congress (from January 3, 2015, to January 3, 2017)

In [24]:
source = requests.get("https://en.wikipedia.org/wiki/114th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link.get('href')) for link in soup.find_all('a')]
begins = stripper(begins)
ends = []
for string in begins:
    if re.match(r"/wiki", string):
        ends.append(re.sub(r"/wiki/","",string))
sens114 = ends[229:379]
reps114 = ends[390:1318]
####
####
####
sstate = []
schamber = []
sname = []
rstate = []
rchamber = []
rname = []
for sen in sens114:
    if re.match(r"List_of.*", sen):
        state = re.sub(r"^List_of_United_States_Senators_from_","",sen)
    else:
        name = re.sub(r"_"," ", sen)
        name = re.sub(r"\(.*\)","",name)
        sname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        sstate.append(state)
        schamber.append("Senate")
for dis in reps114:
    if re.match(r"List_of.*", dis):
        state = re.sub(r"^List_of_United_States_Representatives_from_","",dis)
    #elif re.match(r".*_congressional_district$", dis):
    #    house114.append(state)
    #    temp = re.sub(r"_congressional_district$","",dis)
    #    house114.append("DISTRICT: " + re.sub(r"^.*[^_]+_","", temp))
    elif not re.match(r".*_congressional_district$", dis):
        name = re.sub(r"_"," ",dis)
        name = re.sub(r"\(.*\)","",name)
        rname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        rstate.append(state)
        rchamber.append("House")
rstates114 = pd.concat([pd.Series(rname), pd.Series(rchamber), pd.Series(rstate)], axis = 1)
rstates114.columns = ["Name", "Chamber", "State"] 
sstates114 = pd.concat([pd.Series(sname), pd.Series(schamber), pd.Series(sstate)], axis = 1)
sstates114.columns = ["Name", "Chamber", "State"] 

In [25]:
sstates114.head()

,Name,Chamber,State
0,Jeff Sessions,Senate,Alabama
1,Richard Shelby,Senate,Alabama
2,Dan Sullivan,Senate,Alaska
3,Lisa Murkowski,Senate,Alaska
4,Jeff Flake,Senate,Arizona


In [26]:
sstates114.tail()

,Name,Chamber,State
95,Shelley Moore Capito,Senate,West_Virginia
96,Tammy Baldwin,Senate,Wisconsin
97,Ron Johnson,Senate,Wisconsin
98,John Barrasso,Senate,Wyoming
99,Mike Enzi,Senate,Wyoming


In [27]:
rstates114.head()

,Name,Chamber,State
0,Bradley Byrne,House,Alabama
1,Martha Roby,House,Alabama
2,Mike Rogers,House,Alabama
3,Robert Aderholt,House,Alabama
4,Mo Brooks,House,Alabama


In [28]:
rstates114.tail()

,Name,Chamber,State
437,Jim Sensenbrenner,House,Wisconsin
438,Glenn Grothman,House,Wisconsin
439,Sean Duffy,House,Wisconsin
440,Reid Ribble,House,Wisconsin
441,Cynthia Lummis,House,Wyoming


In [29]:
# 114th Congress (from January 3, 2015, to January 3, 2017)

In [30]:
source = requests.get("https://en.wikipedia.org/wiki/114th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link) for link in soup.find_all('li')]
begins = stripper(begins)
sens114 = begins[223:323]
reps114 = begins[374:818]
syear = ["<BLANK>" for x in range(len(sens114))]
sday = ["<BLANK>" for x in range(len(sens114))]
smonth = ["<BLANK>" for x in range(len(sens114))]
sparty = ["<BLANK>" for x in range(len(sens114))]
sname = ["<BLANK>" for x in range(len(sens114))]
sstate = ["<BLANK>" for x in range(len(sens114))]
##########
ryear = ["<BLANK>" for x in range(len(reps114))]
rday = ["<BLANK>" for x in range(len(reps114))]
rmonth = ["<BLANK>" for x in range(len(reps114))]
rparty = ["<BLANK>" for x in range(len(reps114))]
rname = ["<BLANK>" for x in range(len(reps114))]
rstate = ["<BLANK>" for x in range(len(reps114))]
rdistrict = ["<BLANK>" for x in range(len(reps114))]
hmm = [re.sub(r"<[^>]+>","",sen) for sen in sens114]
for sen in range(len(hmm)):
    z = re.match("^2\..*",hmm[sen]) #class 2 begins this congress
    if z:
        syear[sen] = "2014" #election year
        smonth[sen] = "11" #election month
        sday[sen] = "4" #election day
        name = re.sub("^2\. ","",hmm[sen]) #Senate cycle according to wikipedia metadata
        sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(sname[sen],"",name)
        party = re.findall(r"\([^ ]+\)", party)[0]
        sparty[sen] = re.sub(r"[^A-Z]+","",party)
for sen in range(len(hmm)):
    if syear[sen] == "<BLANK>":
        if re.match("^[^0-9].*",hmm[sen]):
            step2 = re.findall(r"[A-Za-z]+ [0-9]+, \d\d\d\d",hmm[sen])[0]
            sday[sen] = re.sub(r",","",re.findall(r"[0-9]+,",step2)[0])
            syear[sen] = re.findall(r"\d\d\d\d", step2)[0]
            smonth[sen] = mdict[re.sub(r"[0-9, ]+","", step2)]
            name = re.sub(r"\[[^ ]+\]","",hmm[sen])
            sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
            party = re.sub(sname[sen],"",name)
            sparty[sen] = re.sub(r"[^A-Z]+","",party)
syear = [x for x in syear if x!= "<BLANK>"]
smonth = [x for x in smonth if x!= "<BLANK>"]
sday = [x for x in sday if x!= "<BLANK>"]
sname = [x for x in sname if x!= "<BLANK>"]
sparty = [x[0] for x in sparty if x!= "<BLANK>"]
schamber = ["Senate" for x in sparty if x!= "<BLANK>"]
df = pd.DataFrame({'year': syear,
                       'month': smonth,
                       'day': sday})
sdate = pd.to_datetime(df)
senate114 = pd.concat([sdate, pd.Series(sparty), pd.Series(sname), pd.Series(schamber)], axis = 1)
senate114.columns = ["Date", "Party", "Name", "Chamber"]
#######
#######
#######
hmm = [re.sub(r"<[^>]+>","",rep) for rep in reps114]
hmm = [re.sub("[Aa]t\-large\.","0.",rep) for rep in hmm]
for rep in range(len(hmm)):
    z = re.match("^[0-9]+\..*",hmm[rep])
    if z:
        district = re.findall(r"^[0-9]+", hmm[rep])[0]
        rdistrict[rep] = district
        name = re.sub(rdistrict[rep] + ". ","", hmm[rep])
        rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(rname[rep] + " ","",name)
        party = re.findall(r"^\([^ ]+\)",party)[0]
        rparty[rep] = re.sub(r"[\(\)]","", party)
        ryear[rep] = "2014" #election year
        rmonth[rep] = "11" #election month
        rday[rep] = "4" #election day   
    else:
        if not re.search(".*Vacant.*", hmm[rep]):
            rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", hmm[rep])[0]
            rdistrict[rep] = district
            party = re.sub(rname[rep] + " ","", hmm[rep])
            party = re.findall(r"^\([^ ]+\)",party)[0]
            rparty[rep] = re.sub(r"[\(\)]","", party)
            date = re.sub("^.*from ","",hmm[rep])
            ryear[rep] = re.findall(r"\d\d\d\d",date)[0]
            date = re.sub(ryear[rep],"",date)
            date = re.sub(r",.*$","",date)
            rday[rep] = re.findall(r"[0-9]+",date)[0]
            date = re.sub(" " + rday[rep],"",date)
            rmonth[rep] = mdict[date]
ryear = [x for x in ryear if x!= "<BLANK>"]
rmonth = [x for x in rmonth if x!= "<BLANK>"]
rday = [x for x in rday if x!= "<BLANK>"]
rname = [x for x in rname if x!= "<BLANK>"]
rparty = [x[0] for x in rparty if x!= "<BLANK>"]
rchamber = ["House" for x in rparty if x!="<BLANK>"]
rdistrict = [x for x in rdistrict if x!="<BLANK>"]
df = pd.DataFrame({'year': ryear,
                       'month': rmonth,
                       'day': rday})
rdate = pd.to_datetime(df)
house114 = pd.concat([rdate, pd.Series(rparty), pd.Series(rname), pd.Series(rchamber), pd.Series(rdistrict)], axis = 1)
house114.columns = ["Date", "Party", "Name", "Chamber", "District"]   

In [31]:
senate114.head()

,Date,Party,Name,Chamber
0,2014-11-04,R,Jeff Sessions,Senate
1,2014-11-04,R,Dan Sullivan,Senate
2,2014-11-04,R,Tom Cotton,Senate
3,2014-11-04,R,Cory Gardner,Senate
4,2014-11-04,D,Chris Coons,Senate


In [32]:
senate114.tail()

,Date,Party,Name,Chamber
28,2014-11-04,R,Lamar Alexander,Senate
29,2014-11-04,R,John Cornyn,Senate
30,2014-11-04,D,Mark Warner,Senate
31,2014-11-04,R,Shelley Moore Capito,Senate
32,2014-11-04,R,Mike Enzi,Senate


In [33]:
house114.head()

,Date,Party,Name,Chamber,District
0,2014-11-04,R,Bradley Byrne,House,1
1,2014-11-04,R,Martha Roby,House,2
2,2014-11-04,R,Mike Rogers,House,3
3,2014-11-04,R,Robert Aderholt,House,4
4,2014-11-04,R,Mo Brooks,House,5


In [34]:
house114.tail()

,Date,Party,Name,Chamber,District
437,2014-11-04,R,Jim Sensenbrenner,House,5
438,2014-11-04,R,Glenn Grothman,House,6
439,2014-11-04,R,Sean Duffy,House,7
440,2014-11-04,R,Reid Ribble,House,8
441,2014-11-04,R,Cynthia Lummis,House,0


# 115th Congress (from January 3, 2017, to January 3, 2019)

In [35]:
source = requests.get("https://en.wikipedia.org/wiki/115th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link.get('href')) for link in soup.find_all('a')]
begins = stripper(begins)
ends = []
for string in begins:
    if re.match(r"/wiki", string):
        ends.append(re.sub(r"/wiki/","",string))
sens115 = ends[210:365]
reps115 = ends[377:1307]
####
####
####
sstate = []
schamber = []
sname = []
rstate = []
rchamber = []
rname = []
for sen in sens115:
    if re.match(r"List_of.*", sen):
        state = re.sub(r"^List_of_United_States_Senators_from_","",sen)
    else:
        name = re.sub(r"_"," ", sen)
        name = re.sub(r"\(.*\)","",name)
        sname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        sstate.append(state)
        schamber.append("Senate")
for dis in reps115:
    if re.match(r"List_of.*", dis):
        state = re.sub(r"^List_of_United_States_Representatives_from_","",dis)
    #elif re.match(r".*_congressional_district$", dis):
    #    house115.append(state)
    #    temp = re.sub(r"_congressional_district$","",dis)
    #    house115.append("DISTRICT: " + re.sub(r"^.*[^_]+_","", temp))
    elif not re.match(r".*_congressional_district$", dis):
        name = re.sub(r"_"," ",dis)
        name = re.sub(r"\(.*\)","",name)
        rname.append(re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0])
        rstate.append(state)
        rchamber.append("House")
rstates115 = pd.concat([pd.Series(rname), pd.Series(rchamber), pd.Series(rstate)], axis = 1)
rstates115.columns = ["Name", "Chamber", "State"] 
sstates115 = pd.concat([pd.Series(sname), pd.Series(schamber), pd.Series(sstate)], axis = 1)
sstates115.columns = ["Name", "Chamber", "State"] 

In [36]:
sstates115.head()

,Name,Chamber,State
0,Jeff Sessions,Senate,Alabama
1,Luther Strange,Senate,Alabama
2,Doug Jones,Senate,Alabama
3,Richard Shelby,Senate,Alabama
4,Dan Sullivan,Senate,Alaska


In [37]:
sstates115.tail()

,Name,Chamber,State
100,Shelley Moore Capito,Senate,West_Virginia
101,Tammy Baldwin,Senate,Wisconsin
102,Ron Johnson,Senate,Wisconsin
103,John Barrasso,Senate,Wyoming
104,Mike Enzi,Senate,Wyoming


In [38]:
rstates115.head()

,Name,Chamber,State
0,Bradley Byrne,House,Alabama
1,Martha Roby,House,Alabama
2,Mike Rogers,House,Alabama
3,Robert Aderholt,House,Alabama
4,Mo Brooks,House,Alabama


In [39]:
rstates115.tail()

,Name,Chamber,State
440,Jim Sensenbrenner,House,Wisconsin
441,Glenn Grothman,House,Wisconsin
442,Sean Duffy,House,Wisconsin
443,Mike Gallagher,House,Wisconsin
444,Liz Cheney,House,Wyoming


In [40]:
# 115th Congress (from January 3, 2017, to January 3, 2019)

In [41]:
source = requests.get("https://en.wikipedia.org/wiki/115th_United_States_Congress#Members")
soup = bs.BeautifulSoup(source.content, features='html.parser') 
begins = [str(link) for link in soup.find_all('li')]
begins = stripper(begins)
sens115 = begins[257:362]
reps115 = begins[413:865]
syear = ["<BLANK>" for x in range(len(sens115))]
sday = ["<BLANK>" for x in range(len(sens115))]
smonth = ["<BLANK>" for x in range(len(sens115))]
sparty = ["<BLANK>" for x in range(len(sens115))]
sname = ["<BLANK>" for x in range(len(sens115))]
sstate = ["<BLANK>" for x in range(len(sens115))]
##########
ryear = ["<BLANK>" for x in range(len(reps115))]
rday = ["<BLANK>" for x in range(len(reps115))]
rmonth = ["<BLANK>" for x in range(len(reps115))]
rparty = ["<BLANK>" for x in range(len(reps115))]
rname = ["<BLANK>" for x in range(len(reps115))]
rstate = ["<BLANK>" for x in range(len(reps115))]
rdistrict = ["<BLANK>" for x in range(len(reps115))]
hmm = [re.sub(r"<[^>]+>","",sen) for sen in sens115]
for sen in range(len(hmm)):
    z = re.match("^3\..*",hmm[sen]) #class 3 begins this congress
    if z:
        syear[sen] = "2016" #election year
        smonth[sen] = "11" #election month
        sday[sen] = "8" #election day
        name = re.sub("^3\. ","",hmm[sen]) #Senate cycle according to wikipedia metadata
        sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(sname[sen],"",name)
        party = re.findall(r"\([^ ]+\)", party)[0]
        sparty[sen] = re.sub(r"[^A-Z]+","",party)
for sen in range(len(hmm)):
    if syear[sen] == "<BLANK>":
        if re.match("^[^0-9].*",hmm[sen]):
            step2 = re.findall(r"[A-Za-z]+ [0-9]+, \d\d\d\d",hmm[sen])[0]
            sday[sen] = re.sub(r",","",re.findall(r"[0-9]+,",step2)[0])
            syear[sen] = re.findall(r"\d\d\d\d", step2)[0]
            smonth[sen] = mdict[re.sub(r"[0-9, ]+","", step2)]
            name = re.sub(r"\[[^ ]+\]","",hmm[sen])
            sname[sen] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
            party = re.sub(sname[sen],"",name)
            sparty[sen] = re.sub(r"[^A-Z]+","",party)
syear = [x for x in syear if x!= "<BLANK>"]
smonth = [x for x in smonth if x!= "<BLANK>"]
sday = [x for x in sday if x!= "<BLANK>"]
sname = [x for x in sname if x!= "<BLANK>"]
sparty = [x[0] for x in sparty if x!= "<BLANK>"]
schamber = ["Senate" for x in sparty if x!= "<BLANK>"]
df = pd.DataFrame({'year': syear,
                       'month': smonth,
                       'day': sday})
sdate = pd.to_datetime(df)
senate115 = pd.concat([sdate, pd.Series(sparty), pd.Series(sname), pd.Series(schamber)], axis = 1)
senate115.columns = ["Date", "Party", "Name", "Chamber"]
#######
#######
#######
hmm = [re.sub(r"<[^>]+>","",rep) for rep in reps115]
hmm = [re.sub("[Aa]t\-large\.","0.",rep) for rep in hmm]
for rep in range(len(hmm)):
    z = re.match("^[0-9]+\..*",hmm[rep])
    if z:
        district = re.findall(r"^[0-9]+", hmm[rep])[0]
        rdistrict[rep] = district
        name = re.sub(rdistrict[rep] + ". ","", hmm[rep])
        rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", name)[0]
        party = re.sub(rname[rep] + " ","",name)
        party = re.findall(r"^\([^ ]+\)",party)[0]
        rparty[rep] = re.sub(r"[\(\)]","", party)
        ryear[rep] = "2016" #election year
        rmonth[rep] = "11" #election month
        rday[rep] = "8" #election day   
    else:
        if not re.search(".*Vacant.*", hmm[rep]):
            rdistrict[rep] = district
            rname[rep] = re.findall(r"^[A-Za-z \-\.\']+ [A-Za-z\-\'\.]+", hmm[rep])[0]
            party = re.sub(rname[rep] + " ","", hmm[rep])
            party = re.findall(r"^\([^ ]+\)",party)[0]
            rparty[rep] = re.sub(r"[\(\)]","", party)
            date = re.sub("^.*from ","",hmm[rep])
            ryear[rep] = re.findall(r"\d\d\d\d",date)[0]
            date = re.sub(ryear[rep],"",date)
            date = re.sub(r",.*$","",date)
            rday[rep] = re.findall(r"[0-9]+",date)[0]
            date = re.sub(" " + rday[rep],"",date)
            rmonth[rep] = mdict[date]
            rdistrict[rep] = district
ryear = [x for x in ryear if x!= "<BLANK>"]
rmonth = [x for x in rmonth if x!= "<BLANK>"]
rday = [x for x in rday if x!= "<BLANK>"]
rname = [x for x in rname if x!= "<BLANK>"]
rparty = [x[0] for x in rparty if x!= "<BLANK>"]
rchamber = ["House" for x in rparty if x!= "<BLANK>"]
rdistrict = [x for x in rdistrict if x!="<BLANK>"]
df = pd.DataFrame({'year': ryear,
                       'month': rmonth,
                       'day': rday})
rdate = pd.to_datetime(df)
house115 = pd.concat([rdate, pd.Series(rparty), pd.Series(rname), pd.Series(rchamber), pd.Series(rdistrict)], axis = 1)
house115.columns = ["Date", "Party", "Name", "Chamber", "District"]


In [42]:
senate115.head()

,Date,Party,Name,Chamber
0,2017-02-09,R,Luther Strange,Senate
1,2018-01-03,D,Doug Jones,Senate
2,2016-11-08,R,Richard Shelby,Senate
3,2016-11-08,R,Lisa Murkowski,Senate
4,2016-11-08,R,John McCain,Senate


In [43]:
senate115.tail()

,Date,Party,Name,Chamber
34,2016-11-08,R,John Thune,Senate
35,2016-11-08,R,Mike Lee,Senate
36,2016-11-08,D,Patrick Leahy,Senate
37,2016-11-08,D,Patty Murray,Senate
38,2016-11-08,R,Ron Johnson,Senate


In [44]:
house115.head()

,Date,Party,Name,Chamber,District
0,2016-11-08,R,Bradley Byrne,House,1
1,2016-11-08,R,Martha Roby,House,2
2,2016-11-08,R,Mike Rogers,House,3
3,2016-11-08,R,Robert Aderholt,House,4
4,2016-11-08,R,Mo Brooks,House,5


In [45]:
house115.tail()

,Date,Party,Name,Chamber,District
440,2016-11-08,R,Jim Sensenbrenner,House,5
441,2016-11-08,R,Glenn Grothman,House,6
442,2016-11-08,R,Sean Duffy,House,7
443,2016-11-08,R,Mike Gallagher,House,8
444,2016-11-08,R,Liz Cheney,House,0


# Mighty Joins !

In [46]:
def checker(x,y):
    okay  = pd.merge(x, y,  how='outer', left_on=['Name','Chamber'], right_on = ['Name','Chamber'])
    weird = okay[okay.isnull().any(axis=1)]
    return(len(weird))
def wurry(x,y):
    okay  = pd.merge(x, y,  how='outer', left_on=['Name','Chamber'], right_on = ['Name','Chamber'])
    weird = okay[okay.isnull().any(axis=1)]
    return(weird)
def okee(x,y):
    okay  = pd.merge(x, y,  how='inner', left_on=['Name','Chamber'], right_on = ['Name','Chamber'])
    return(len(okay))
def hurry(x,y):
    return(pd.merge(x, y,  how='inner', left_on=['Name','Chamber'], right_on = ['Name','Chamber']))


In [47]:
senate = pd.concat([hurry(senate115,sstates115),\
                     hurry(senate114,sstates114),\
                     hurry(senate113,sstates113),\
                     hurry(senate112,sstates112)], axis = 0)
senate = senate.reset_index(drop = True)
senate["State"] = senate["State"].str.replace("_"," ")
senate["State"] = senate["State"].map(STATES_DICT)
senate["Party"] = senate["Party"].str.lower()
senate["Name"] = senate["Name"].str.lower()
senate["Name"] = senate["Name"].str.replace(r"[^a-z ]","")
senate["Chamber"] = senate["Chamber"].str.lower()
senate.columns = ["date", "party", "name", "chamber", "state"]
senate = senate.drop_duplicates()
######
######
######
house = pd.concat([hurry(house115,rstates115),\
                     hurry(house114,rstates114),\
                     hurry(house113,rstates113),\
                     hurry(house112,rstates112)], axis = 0)
house = house.reset_index(drop = True)
house["State"] = house["State"].str.replace("_"," ")
house["State"] = house["State"].map(STATES_DICT)
house["Party"] = house["Party"].str.lower()
house["Name"] = house["Name"].str.lower()
house["Name"] = house["Name"].str.replace(r"[^a-z ]","")
house["Chamber"] = house["Chamber"].str.lower()
house.columns = ["date", "party", "name", "chamber", "district", "state"]
house = house[["date", "party", "name", "chamber","state", "district"]]
house = house.drop_duplicates()

In [48]:
house.head()

,date,party,name,chamber,state,district
0,2016-11-08,r,bradley byrne,house,AL,1
1,2016-11-08,r,martha roby,house,AL,2
2,2016-11-08,r,mike rogers,house,AL,3
3,2016-11-08,r,robert aderholt,house,AL,4
4,2016-11-08,r,mo brooks,house,AL,5


In [49]:
house.tail()

,date,party,name,chamber,state,district
1721,2010-11-02,r,jim sensenbrenner,house,WI,5
1722,2010-11-02,r,tom petri,house,WI,6
1723,2010-11-02,r,sean duffy,house,WI,7
1724,2010-11-02,r,reid ribble,house,WI,8
1725,2010-11-02,r,cynthia lummis,house,WY,0


In [50]:
len(house)

1721

In [51]:
senate.head()

,date,party,name,chamber,state
0,2017-02-09,r,luther strange,senate,AL
1,2018-01-03,d,doug jones,senate,AL
2,2016-11-08,r,richard shelby,senate,AL
3,2016-11-08,r,lisa murkowski,senate,AK
4,2016-11-08,r,john mccain,senate,AZ


In [52]:
senate.tail()

,date,party,name,chamber,state
142,2010-11-02,r,john thune,senate,SD
143,2010-11-02,r,mike lee,senate,UT
144,2010-11-02,d,patrick leahy,senate,VT
145,2010-11-02,d,patty murray,senate,WA
146,2010-11-02,r,ron johnson,senate,WI


In [53]:
len(senate)

147

In [54]:
len(senate) + len(house)

1868

In [57]:
#house.to_csv("Clean_House_Winners.csv")
#senate.to_csv("Clean_Senate_Winners.csv")